In [1]:
import numpy as np
import pandas as pd
import itertools

In [2]:
negative_set = pd.read_csv('negative_set.bed', delimiter='\t', header = None)
gm = pd.read_csv('GM12878.bed', delimiter='\t', header=None)

In [3]:
negative_set.head()

,0,1,2
0,chr10,32251,36771
1,chr10,39431,39891
2,chr10,72312,74222
3,chr10,84717,85177
4,chr10,90499,91949


In [4]:
gm.head()

,0,1,2,3
0,chr1,773300,774100,7.866088
1,chr1,778980,779450,6.472419
2,chr1,800100,802000,11.010675
3,chr1,825670,826410,6.114487
4,chr1,839470,842590,8.848865


In [5]:
train_neg = negative_set.loc[~negative_set[0].isin(['chrY', 'chrX'])]

In [7]:
GRCh37 = pd.read_csv('GRCh37.primary_assembly.genome.fa', sep='\t')

In [8]:
GRCh37_1 = GRCh37.loc[~GRCh37['>chr1 1'].str.contains('NN')]

In [10]:
GRCh37_1

,>chr1 1
167,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC...
168,ACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCT...
169,CCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAACCCT...
170,CCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCC...
171,TAACCCTAACCCTAACCCTAACCCTAACCCCAACCCCAACCCCAAC...
...,...
51696864,GTTGTAGTGCTGGCTGTTGTTGTGGTTGTGGGTTCAGTAGAGGTGG...
51696865,GTGGTTGTGGTCTCAGTAGCAGTGGCTGCAGTTGTGGTGGCAGACT...
51696866,CTTGCTGTGGTTGTCATTGTGGGTTCAGCAGAAGTGGCAGTGGTGG...
51696867,GTCTCAGTGGAAGTGCCTGCAGTTGTGGTGGCAGACTGTTCAGTTG...


In [30]:
def Find_Kmers(string, k):    
    kmers = itertools.product('ACTG', repeat=k)
    for i in kmers:
        if string.count(''.join(i)) > 0:
            print(f"{''.join(i)}: {string.count(''.join(i))}")


In [23]:
Find_Kmers('ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC', 3)

AAC: 7
ACC: 8
CCC: 8
CCT: 7
CTA: 7
TAA: 7


In [83]:
def Kmers_cols(column, k):
    kmers = itertools.product('ACTG', repeat=k)
    df = pd.DataFrame()
    for i in kmers:
        kmer = ''.join(i)
        rev_kmer = kmer.replace('A', '%temp%').replace('T', 'A').replace('%temp%', 'T').replace('C', '%temp%').replace('G', 'C').replace('%temp%', 'G')
        if(rev_kmer not in df.columns):
            df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
        else:
            df[rev_kmer] += column.str.count(kmer)/(len(column.iloc[0])-k+1)
    return df

In [88]:
df_3mers = Kmers_cols(GRCh37_1.head(100)['>chr1 1'], 4)
df_3mers.head()

/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

,AAAA,AAAC,AAAT,AAAG,AACA,AACC,AACT,AACG,AATA,AATC,...,CGCT,CGCG,CGTA,CGTC,CGTT,CGTG,CGGA,CGGC,CGGT,CGGG
167,0.0,0.000000,0.0,0.0,0.0,0.157895,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
168,0.0,0.000000,0.0,0.0,0.0,0.157895,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169,0.0,0.000000,0.0,0.0,0.0,0.157895,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
170,0.0,0.035088,0.0,0.0,0.0,0.157895,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171,0.0,0.000000,0.0,0.0,0.0,0.175439,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
Kmers_cols(GRCh37_1.head(100)['>chr1 1'], 4)

/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[kmer] = column.str.count(kmer)/(len(column.iloc[0])-k+1)
/var/folders/dc/tm97_rl10kj62c4gsppshgm80000gn/T/ipykernel_8546/3841867262.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

,AAAA,AAAC,AAAT,AAAG,AACA,AACC,AACT,AACG,AATA,AATC,...,CGCT,CGCG,CGTA,CGTC,CGTT,CGTG,CGGA,CGGC,CGGT,CGGG
167,0.0,0.000000,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
168,0.0,0.000000,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
169,0.0,0.000000,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
170,0.0,0.035088,0.0,0.000000,0.000000,0.157895,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
171,0.0,0.000000,0.0,0.000000,0.000000,0.175439,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.017544,0.0,0.0,...,0.017544,0.000000,0.0,0.000000,0.017544,0.0,0.000000,0.000000,0.000000,0.035088
263,0.0,0.000000,0.0,0.017544,0.000000,0.000000,0.0,0.017544,0.0,0.0,...,0.035088,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.017544
264,0.0,0.000000,0.0,0.000000,0.000000,0.035088,0.0,0.017544,0.0,0.0,...,0.000000,0.017544,0.0,0.035088,0.000000,0.0,0.035088,0.017544,0.000000,0.017544
265,0.0,0.000000,0.0,0.000000,0.000000,0.017544,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017544,0.017544,0.035088,0.017544
